<a href="https://colab.research.google.com/github/aljoharas/datascience/blob/main/spotifyproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Collection

In [16]:
%env SPOTIPY_CLIENT_ID=091a90ce07be44b2b4bb6f6fcf08fd31
%env SPOTIPY_CLIENT_SECRET=3333569ba9d44fc48d0b4d1e422db526


env: SPOTIPY_CLIENT_ID=091a90ce07be44b2b4bb6f6fcf08fd31
env: SPOTIPY_CLIENT_SECRET=3333569ba9d44fc48d0b4d1e422db526


In [14]:
!pip install spotipy


In [17]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)


In [ ]:
import os
import requests

# List of CSV files in your GitHub repo
csv_urls = [
    "https://raw.githubusercontent.com/yourusername/yourrepo/main/regional-global-weekly-2025-09-18.csv",
    "https://raw.githubusercontent.com/yourusername/yourrepo/main/regional-global-daily-2025-08-28.csv",
    # add all files here
]

# Download files to /content
for url in csv_urls:
    filename = os.path.join("/content", url.split("/")[-1])
    if not os.path.exists(filename):
        r = requests.get(url)
        with open(filename, "wb") as f:
            f.write(r.content)

# Now your glob will work as usual
import glob
chart_files = glob.glob("/content/regional-*.csv")
print(f"Found {len(chart_files)} chart files")

In [19]:
import glob
import pandas as pd

chart_files = glob.glob("/content/regional-*.csv")
print(f"Found {len(chart_files)} chart files")

dfs = [pd.read_csv(f) for f in chart_files]
all_artists = pd.concat([df['artist_names'] for df in dfs]).unique()

print(f"Total unique artists across all charts: {len(all_artists)}")


artist_data = []
seen_ids = set()

for artist_name in all_artists:
    result = sp.search(q=f"artist:{artist_name}", type="artist", limit=1)
    if result['artists']['items']:
        a = result['artists']['items'][0]
        if not a['genres'] or a['id'] in seen_ids:
            continue
        artist_data.append({
            "name": a['name'],
            "id": a['id'],
            "genres": a['genres'],
            "popularity": a['popularity']
        })
        seen_ids.add(a['id'])


# Create DataFrame
df_artists = pd.DataFrame(artist_data)
print(f"✅ Final dataset: {len(df_artists)} artists with non-empty genres")
display(df_artists)


Found 7 chart files
Total unique artists across all charts: 538
✅ Final dataset: 177 artists with non-empty genres


,name,id,genres,popularity
0,Ravyn Lenae,5RTLRtXjbXI2lSXc6jxlAz,"[alternative r&b, indie soul]",80
1,Fleetwood Mac,08GQAI4eElDnROBrJRGE0X,"[classic rock, yacht rock, soft rock]",87
2,Sabrina Carpenter,74KM79TiuVKeVCqs8QtB0B,[pop],96
3,Zach Bryan,40ZNYROS4zLfyyBSs2PGe2,"[country, red dirt]",89
4,Olivia Dean,00x1fYSGhdqScXBRpSj3DW,[pop soul],87
...,...,...,...,...
172,Whiskey Myers,26opZSJcXshCmCwxgZQmBc,"[red dirt, texas country, southern rock, outla...",72
173,"Earth, Wind & Fire",4QQgXkCYTt3BlENzhyNETg,"[funk, disco]",77
174,Tyler Childers,13ZEDW6vyBF12HYcZRr4EV,"[country, outlaw country, red dirt, alt countr...",80
175,Bailey Zimmerman,3win9vGIxFfBRag9S63wwf,[country],80
